# Lab: Transfer Learning

training a network with ImageNet pre-trained weights as a base, but with additional network layers of your own added on. You'll also get to see the difference between using frozen weights and training on all layers.
使用ImageNet预先训练的权值作为基础来训练网络，但是要添加您自己的额外网络层。你也会看到在所有层面上使用冻结权重和在所有层上训练的区别。


冻结权重
当只对模型进行微调时，通常使用冻结权值，因为在训练期间，反向传播和权值更新不会应用于任何冻结层。如果你有一个ImageNet pre-trained模型,大部分的网络可能适用于你的情况,所以你可能只需要切断顶部全层,冻结所有其他层,最后添加一个或多个层不冻执行一些微调。
还有一个不冻结权重的选项，它将在ImageNet预训练的权重(如果适用)上启动您的模型，然后从那里执行进一步的训练。
冻结的一个额外好处的权重也会在形式的内存使用情况,训练速度——VGG等更大的网络,有一个大大大内存使用和较慢的速度,当它需要执行反向传播和重量更新所有层上而不是一小部分(可能性较小)层。
注意:如果您遇到问题，可以通过单击工作区左上角的Jupyter徽标找到一个解决方案笔记本。

In [2]:
# Set a couple flags for training - you can ignore these for now
freeze_flag = True  # `True` to freeze layers, `False` for full training
weights_flag = 'imagenet' # 'imagenet' or None
preprocess_flag = True # Should be true for ImageNet pre-trained typically

# Loads in InceptionV3
from keras.applications.inception_v3 import InceptionV3

# We can use smaller than the default 299x299x3 input for InceptionV3
# which will speed up training. Keras v2.0.9 supports down to 139x139x3
input_size = 139

# Using Inception with ImageNet pre-trained weights
inception = InceptionV3(weights=weights_flag, include_top=False,
                        input_shape=(input_size,input_size,3))

Using TensorFlow backend.


We'll use Inception V3 for this lab, although you can use the same techniques with any of the models in [Keras Applications](https://keras.io/applications/). Do note that certain models are only available in certain versions of Keras; this workspace uses Keras v2.0.9, for which you can see the available models [here](https://faroit.github.io/keras-docs/2.0.9/applications/).

In the above, we've set Inception to use an `input_shape` of 139x139x3 instead of the default 299x299x3. This will help us to speed up our training a bit later (and we'll actually be upsampling from smaller images, so we aren't losing data here). In order to do so, we also must set `include_top` to `False`, which means the final fully-connected layer with 1,000 nodes for each ImageNet class is dropped, as well as a Global Average Pooling layer.

### Pre-trained with frozen weights
To start, we'll see how an ImageNet pre-trained model with all weights frozen in the InceptionV3 model performs. We will also drop the end layer and append new layers onto it, although you could do this in different ways (not drop the end and add new layers, drop more layers than we will here, etc.).
首先，我们将看到在InceptionV3模型中冻结了所有权重的ImageNet预训练模型是如何执行的。我们还将删除结束层并在其上添加新层，尽管您可以使用不同的方法(不是删除结束层并添加新层，而是删除比这里更多的层，等等)。

You can freeze layers by setting `layer.trainable` to False for a given `layer`. Within a `model`, you can get the list of layers with `model.layers`.

In [4]:
if freeze_flag == True:
    ## TODO: Iterate through the layers of the Inception model
    ##       loaded above and set all of them to have trainable = False
    for layer in inception.layers:
        layer.trainable = False

### Dropping layers
You can drop layers from a model with `model.layers.pop()`. Before you do this, you should check out what the actual layers of the model are with Keras's `.summary()` function.

In [5]:
## TODO: Use the model summary function to see all layers in the
##       loaded Inception model
inception.summary

<bound method Container.summary of <keras.engine.training.Model object at 0x00000217381B5470>>

In a normal Inception network, you would see from the model summary that the last two layers were a global average pooling layer, and a fully-connected "Dense" layer. However, since we set `include_top` to `False`, both of these get dropped. If you otherwise wanted to drop additional layers, you would use:

```
inception.layers.pop()
```

Note that `pop()` works from the end of the model backwards.

It's important to note two things here:
1. How many layers you drop is up to you, typically. We dropped the final two already by setting `include_top` to False in the original loading of the model, but you could instead just run `pop()` twice to achieve similar results. (*Note:* Keras requires us to set `include_top` to False in order to change the `input_shape`.) Additional layers could be dropped by additional calls to `pop()`.
2. If you make a mistake with `pop()`, you'll want to reload the model. If you use it multiple times, the model will continue to drop more and more layers, so you may need to check `model.summary()` again to check your work.

### Adding new layers

Now, you can start to add your own layers. While we've used Keras's `Sequential` model before for simplicity, we'll actually use the [Model API](https://keras.io/models/model/) this time. This functions a little differently, in that instead of using `model.add()`, you explicitly tell the model which previous layer to attach to the current layer. This is useful if you want to use more advanced concepts like [skip layers](https://en.wikipedia.org/wiki/Residual_neural_network), for instance (which were used heavily in ResNet).

For example, if you had a previous layer named `inp`:
```
x = Dropout(0.2)(inp)
```
is how you would attach a new dropout layer `x`, with it's input coming from a layer with the variable name `inp`.

We are going to use the [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html), which consists of 60,000 32x32 images of 10 classes. We need to use Keras's `Input` function to do so, and then we want to re-size the images up to the `input_size` we specified earlier (139x139).

In [10]:
from keras.layers import Input, Lambda
import tensorflow as tf

# Makes the input placeholder layer 32x32x3 for CIFAR-10
cifar_input = Input(shape=(32,32,3))

# Re-sizes the input with Kera's Lambda layer & attach to cifar_input
resized_input = Lambda(lambda image: tf.image.resize_images( 
    image, (input_size, input_size)))(cifar_input)

# Feeds the re-sized input into Inception model
# You will need to update the model name if you changed it earlier!
inp = inception(resized_input)


In [13]:
# Imports fully-connected "Dense" layers & Global Average Pooling
from keras.layers import Dense, GlobalAveragePooling2D

## TODO: Setting `include_top` to False earlier also removed the
##       GlobalAveragePooling2D layer, but we still want it.
##       Add it here, and make sure to connect it to the end of Inception
x = GlobalAveragePooling2D()(inp)

## TODO: Create two new fully-connected layers using the Model API
##       format discussed above. The first layer should use `out`
##       as its input, along with ReLU activation. You can choose
##       how many nodes it has, although 512 or less is a good idea.
##       The second layer should take this first layer as input, and
##       be named "predictions", with Softmax activation and 
##       10 nodes, as we'll be using the CIFAR10 dataset.
#x = Dense(512, activation = 'relu')(x)
predictions = Dense(10, activation = 'softmax')(x)

We're almost done with our new model! Now we just need to use the actual Model API to create the full model.

In [14]:
# Imports the Model API
from keras.models import Model

# Creates the model, assuming your final layer is named "predictions"
model = Model(inputs=cifar_input, outputs=predictions)

# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Check the summary of this new model to confirm the architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
lambda_4 (Lambda)            (None, 139, 139, 3)       0         
_________________________________________________________________
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_3 ( (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                20490     
Total params: 21,823,274
Trainable params: 20,490
Non-trainable params: 21,802,784
_________________________________________________________________


Great job creating a new model architecture from Inception! Notice how this method of adding layers before InceptionV3 and appending to the end of it made InceptionV3 condense down into one line in the summary; if you use the Inception model's normal input (which you could gather from `inception.layers.input`), it would instead show all the layers like before.

Most of the rest of the code in the notebook just goes toward loading our data, pre-processing it, and starting our training in Keras, although there's one other good point to make here - Keras callbacks.
从一开始就创建一个新的模型体系结构是一项伟大的工作!请注意，在InceptionV3之前添加层并将其追加到末尾的方法是如何使InceptionV3在摘要中浓缩为一行的;如果您使用Inception模型的普通输入(您可以从init .layers.input中收集它)，它将会像以前一样显示所有的层。
笔记本中剩下的大部分代码只是用来加载数据、对其进行预处理，并开始在Keras中进行培训，不过这里还有一个很好的地方需要说明——Keras回调。

### Keras Callbacks
Keras [callbacks](https://keras.io/callbacks/) allow you to gather and store additional information during training, such as the best model, or even stop training early if the validation accuracy has stopped improving. These methods can help to avoid overfitting, or avoid other issues.
Keras回调允许您在培训期间收集和存储额外的信息，例如最佳模型，或者如果验证精度停止提高，甚至可以提前停止培训。这些方法可以帮助避免过拟合，或避免其他问题。
There's two key callbacks to mention here, `ModelCheckpoint` and `EarlyStopping`. As the names may suggest, model checkpoint saves down the best model so far based on a given metric, while early stopping will end training before the specified number of epochs if the chosen metric no longer improves after a given amount of time.

这里要提到两个关键的回调，ModelCheckpoint和early stop。顾名思义，模型检查点根据给定的指标保存到目前为止最好的模型，而如果所选的指标在给定的时间后不再改善，那么早期停止将在指定的时间点之前结束培训。

To set these callbacks, you could do the following:

要设置这些回调，可以执行以下操作:
```
checkpoint = ModelCheckpoint(filepath=save_path, monitor='val_loss', save_best_only=True)
```
This would save a model to a specified `save_path`, based on validation loss, and only save down the best models. If you set `save_best_only` to `False`, every single epoch will save down another version of the model.

这将根据验证损失将模型保存到指定的save_path，并且只保存最好的模型。如果您将save_best_only设置为False，那么每个历元都将保存模型的另一个版本。

```
stopper = EarlyStopping(monitor='val_acc', min_delta=0.0003, patience=5)
```
This will monitor validation accuracy, and if it has not decreased by more than 0.0003 from the previous best validation accuracy for 5 epochs, training will end early.
这将监控验证的准确性，如果它没有比之前5个时期的最佳验证精度下降超过0.0003，那么培训将提前结束。

You still need to actually feed these callbacks into `fit()` when you train the model (along with all other relevant data to feed into `fit`):
当您训练模型时(连同所有其他相关数据一起)，您仍然需要实际地将这些回调输入fit():
```
model.fit(callbacks=[checkpoint, stopper])
```

In [8]:
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from keras.datasets import cifar10

(X_train, y_train), (X_val, y_val) = cifar10.load_data()

# One-hot encode the labels
label_binarizer = LabelBinarizer()
y_one_hot_train = label_binarizer.fit_transform(y_train)
y_one_hot_val = label_binarizer.fit_transform(y_val)

# Shuffle the training & test data
X_train, y_one_hot_train = shuffle(X_train, y_one_hot_train)
X_val, y_one_hot_val = shuffle(X_val, y_one_hot_val)

# We are only going to use the first 10,000 images for speed reasons
# And only the first 2,000 images from the test set
X_train = X_train[:2000]
y_one_hot_train = y_one_hot_train[:2000]
X_val = X_val[:400]
y_one_hot_val = y_one_hot_val[:400]

You can check out Keras's [ImageDataGenerator documentation](https://faroit.github.io/keras-docs/2.0.9/preprocessing/image/) for more information on the below - you can also add additional image augmentation through this function, although we are skipping that step here so you can potentially explore it in the upcoming project.

In [9]:
# Use a generator to pre-process our images for ImageNet
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input

if preprocess_flag == True:
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
else:
    datagen = ImageDataGenerator()
    val_datagen = ImageDataGenerator()

In [10]:
# Train the model
batch_size = 32
epochs = 5
# Note: we aren't using callbacks here since we only are using 5 epochs to conserve GPU time
model.fit_generator(datagen.flow(X_train, y_one_hot_train, batch_size=batch_size), 
                    steps_per_epoch=len(X_train)/batch_size, epochs=epochs, verbose=1, 
                    validation_data=val_datagen.flow(X_val, y_one_hot_val, batch_size=batch_size),
                    validation_steps=len(X_val)/batch_size)

Epoch 1/5
63/62 [==============================] - 233s - loss: 1.7256 - acc: 0.4137 - val_loss: 1.1955 - val_acc: 0.6650
Epoch 2/5
63/62 [==============================] - 228s - loss: 1.1022 - acc: 0.6409 - val_loss: 1.1578 - val_acc: 0.6200
Epoch 3/5
63/62 [==============================] - 226s - loss: 0.9580 - acc: 0.6835 - val_loss: 1.0928 - val_acc: 0.6500
Epoch 4/5
63/62 [==============================] - 226s - loss: 0.8361 - acc: 0.7187 - val_loss: 1.0487 - val_acc: 0.6675
Epoch 5/5
63/62 [==============================] - 225s - loss: 0.7097 - acc: 0.7684 - val_loss: 1.1868 - val_acc: 0.6425


As you may have noticed, CIFAR-10 is a fairly tough dataset. However, given that we are only training on a small subset of the data, only training for five epochs, and not using any image augmentation, the results are still fairly impressive!

We achieved ~70% validation accuracy here, although your results may vary.
正如您可能已经注意到的，CIFAR-10是一个相当严格的数据集。然而，考虑到我们只对一小部分数据进行了训练，只对5个纪元进行了训练，并且没有使用任何图像增强，结果仍然相当令人印象深刻!
我们在这里达到了~70%的验证精度，尽管您的结果可能有所不同

## [Optional] Test without frozen weights, or by training from scratch.
没有冻结权重的训练，或者从头开始训练。

Since the majority of the model was frozen above, training speed is pretty quick. You may also want to check out the training speed, as well as final accuracy, if you don't freeze the weights. Note that this can be fairly slow, so we're marking this as optional in order to conserve GPU time. 

If you do want to see the results from doing so, go back to the first code cell and set `freeze_flag` to `False`. If you want to completely train from scratch without ImageNet pre-trained weights, follow the previous step as well as setting `weights_flag` to `None`. Then, go to *Kernel > Restart & Run All*.
由于上面大部分模型都是冻结的，所以训练速度非常快。如果你不冻结权重，你也可以检查训练的速度，以及最终的准确性。注意，这可能相当慢，因此我们将其标记为可选的，以节省GPU时间。
如果您确实希望看到这样做的结果，请返回到第一个代码单元格，并将freeze_flag设置为False。如果您想在没有ImageNet预训练权值的情况下完全从头开始训练，请遵循前面的步骤，并将weights_flag设置为None。然后，转到内核>重新启动并运行所有。

## Comparison

So that you don't use up your GPU time, we've tried out these results ourselves as well.
为了不让你浪费你的GPU时间，我们自己也尝试了这些结果。

Training Mode | Val Acc @ 1 epoch | Val Acc @ 5 epoch | Time per epoch
---- | :----: | :----: | ----:
Frozen weights | 65.5% | 70.3% | 50 seconds
Unfrozen weights | 50.6% | 71.6% | 142 seconds
No pre-trained weights | 19.2% | 39.2% | 142 seconds

From the above, we can see that the pre-trained model with frozen weights actually began converging the fastest (already at 65.5% after 1 epoch), while the model re-training from the pre-trained weights slightly edged it out after 5 epochs.

从上面可以看出，权重冻结的预训练模型实际上收敛速度最快(1个纪元后已经达到65.5%)，而权重冻结的模型在5个纪元后再训练速度略快。

However, this does not tell the whole story - the training accuracy was substantially higher, nearing 87% for the unfrozen weights model. It actually began overfit the data much more under this method. We would likely be able to counteract some of this issue by using data augmentation. On the flip side, the model using frozen weights could also have been improved by actually only freezing a portion of the weights; some of these are likely more specific to ImageNet classes as it gets later in the network, as opposed to the simpler features extracted early in the network.

然而，这并不能说明问题的全部——在未冻结的重量模型中，训练的准确性要高得多，接近87%。实际上，在这种方法下，它开始过度拟合数据。我们可能可以通过使用数据扩充来抵消其中的一些问题。另一方面，使用冷冻权值的模型也可以通过只冷冻部分权值来改进;随着ImageNet类在网络中使用的时间越来越晚，其中一些特性可能更加特定于它，而不是在网络早期提取的更简单的特性。

### The Power of Transfer Learning
Comparing the last line to the other two really shows the power of transfer learning. After five epochs, a model without ImageNet pre-training had only achieved 39.2% accuracy, compared to over 70% for the other two. As such, pre-training the network has saved substantial time, especially given the additional training time needed when the weights are not frozen.

There is also evidence found in various research that pre-training on ImageNet weights will result in a higher overall accuracy than completely training from scratch, even when using a substantially different dataset.